# Scrittopoli - Giornata 2

In [1]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk
import matplotlib.mlab as mlab

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv
import string

import mylib.spreadsheet as ms
import mylib.elo as melo
import mylib.utility as mu


In [2]:
# Generale

day="20171214_142354" # definitivo
calendario_csv="data/calendario_%s.csv" % day
gironi_csv="data/gironi_%s.csv" % day
tabellone_csv="data/tabellone_%s.csv" % day
ranking_csv="data/ranking_%s.csv" % day
scrittopoli_csv="data/scrittopoli_%s.csv" % day
elo_giocatori_csv="data/elo_giocatori_%s.csv" % day
elo_squadre_csv="data/elo_squadre_%s.csv" % day

calendario=pd.read_csv(calendario_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()
calendario=calendario.set_index(["girone","partita"], append=True)

gironi=pd.read_csv(gironi_csv,sep=":",quotechar='"',header=0)

tabellone=pd.read_csv(tabellone_csv,sep=",",quotechar='"',header=0,index_col=0)
elo_giocatori=pd.read_csv(elo_giocatori_csv,sep=",",quotechar='"',header=0,index_col=0)

ranking=pd.read_csv(ranking_csv,sep=",",quotechar='"',header=0)
ranking=ranking[ranking["squadra"]!="(riposo)"]
ranking=ranking.set_index("squadra")

scrittopoli=pd.read_csv(scrittopoli_csv,sep=",",quotechar='"',header=0)
scrittopoli=scrittopoli.set_index(["squadra","giocatore"])

elo_squadre=pd.read_csv(elo_squadre_csv,sep=",",quotechar='"',header=0)
elo_squadre=elo_squadre.set_index(["squadra","giocatore"])

# Giornata 1

label="giornata_1"
schedina_csv="data/schedina_"+label+".csv"
classifica_csv="data/classifica_"+label+".csv"
elo_g_csv="data/elo_g_"+label+".csv"
elo_s_csv="data/elo_s_"+label+".csv"

schedina_g1=pd.read_csv(schedina_csv,sep=",",quotechar='"')
classifica_g1=pd.read_csv(classifica_csv,sep=",",quotechar='"').set_index(["girone","squadra"])
elo_g_g1=pd.read_csv(elo_g_csv,sep=",",quotechar='"').set_index("giocatore")
elo_s_g1=pd.read_csv(elo_s_csv,sep=",",quotechar='"').set_index("squadra")

GIORNATA=2

## Come si svolgono gli incontri

Ogni capitano presenta la formazione (due titolari e una riserva). I due titolari vengono accoppiati agli avversari sulla base del punteggio Elo (i due più alti tra loro e i due più bassi tra loro).

Inseriscono i due racconti. Cose da controllare:
* link al racconto;
* tag;
* lunghezza del racconto;
* link al commento;
* commento valido.

Per la votazione ogni squadra presenta l'elenco dei voti da dare a quelle degli altri gironi. Per ogni partita danno quattro punteggi. Oltre a queste ci sono le penalità dell'arbitro.


In [3]:
quadro=calendario.loc[GIORNATA].copy()

riposano=quadro[quadro["squadra 2"]=="(riposo)"][["squadra 1"]]
riposano




,,squadra 1
girone,partita,
A,2,La threesome


In [4]:
quadro=quadro[quadro["squadra 2"]!="(riposo)"]
quadro["vs"]="contro"

quadro=quadro[["squadra 1","vs","squadra 2"]]

quadro=quadro.sort_index()
quadro

squadra 1      vs                      squadra 2
girone partita                                                           
A      1               D Euforiche  contro                     Gli ultimi
B      1               Green House  contro  Gli scrittori del quartierino
       2                I Bananari  contro                 the 3 BrOthers
C      1             Living Stones  contro                    Le TreMende
       2        Retroguardia Trash  contro             Armata Brancaleone

In [5]:
importlib.reload(mu)
mu.to_table_html_multi(quadro)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>squadra 1</th>
<th>vs</th>
<th>squadra 2</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli ultimi</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>I Bananari</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>the 3 BrOthers</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Le TreMende</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Retroguardia Trash</td>
<td style='text-a

## Fase 1. Dalle formazioni ai match

In [6]:
squadre=scrittopoli.copy()
squadre=squadre.reset_index()[["squadra","giocatore"]].set_index("squadra")

importlib.reload(ms)
s=ms.N2017Spreadsheet()

old_data=s.get_formazioni(GIORNATA)
s.set_formazioni(GIORNATA,riposano["squadra 1"].values,squadre.index.unique(),old_data)


#formazioni=s.get_formazioni(GIORNATA)
#formazioni



In [7]:
# Versione random di prova, da sostituire con lettura della formazione via spreadsheet sotto
#data=[]
#for sq in squadre.index.unique():
#    capitano=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    riserva=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    titolari=list(squadre[squadre["giocatore"]!=riserva].loc[sq]["giocatore"])
#    data.append([sq,capitano,riserva]+titolari)
#formazioni=pd.DataFrame(data,columns=["squadra","capitano","riserva","titolare 1","titolare 2"])
#formazioni=formazioni.set_index("squadra")
#formazioni["riposa"]=""
#formazioni.set_value(riposano.iloc[0]["squadra 1"],"riposa","(riposa)")

formazioni=s.get_formazioni(GIORNATA)


In [8]:
formazioni

,riposa,capitano,riserva,titolare 1,titolare 2
squadra,,,,,
Green House,,Bango Skank,Bango Skank,Joyopi,Plata
La threesome,(riposa),mina99,,,
I Bananari,,simone volponi,massimopud,Niko,simone volponi
Armata Brancaleone,,AdStr,Skorpiòs,Ella F.,AdStr
Living Stones,,Macleo,Vincenzo Iennaco,Macleo,Ljuset
Retroguardia Trash,,Andrea28,AlexComan,Eudes,Andrea28
Le TreMende,,Emy,sefora,Emy,Thea
Gli scrittori del quartierino,,Marty12,Federico72,camparino,Marty12
the 3 BrOthers,,AryaSophia,ettore70,ITG,AryaSophia


In [9]:
importlib.reload(melo)

elo_g1=elo_g_g1.copy()
elo_g1.columns=["squadra","rank 1"]

# c'è una componente casuale, una volta pubblicati vanno fissati
#accoppiamenti=melo.calcola_accoppiamenti(GIORNATA,formazioni,calendario,elo_g1)
# accoppiamenti.reset_index().as_matrix()

data=[['A', 1, 'D Euforiche', 'Kuno', 'Marcello', 'Kuno', 'queffe'],
       ['A', 1, 'Gli ultimi', 'crazycat', 'Ghigo', 'caipiroska', 'crazycat'],
       ['B', 1, 'Green House', 'Bango Skank', 'Bango Skank', 'Plata',
        'Joyopi'],
       ['B', 1, 'Gli scrittori del quartierino', 'Marty12', 'Federico72',
        'camparino', 'Marty12'],
       ['B', 2, 'I Bananari', 'simone volponi', 'massimopud',
        'simone volponi', 'Niko'],
       ['B', 2, 'the 3 BrOthers', 'AryaSophia', 'ettore70', 'ITG',
        'AryaSophia'],
       ['C', 1, 'Living Stones', 'Macleo', 'Vincenzo Iennaco', 'Ljuset',
        'Macleo'],
       ['C', 1, 'Le TreMende', 'Emy', 'sefora', 'Thea', 'Emy'],
       ['C', 2, 'Retroguardia Trash', 'Andrea28', 'AlexComan', 'Andrea28',
        'Eudes'],
       ['C', 2, 'Armata Brancaleone', 'AdStr', 'Skorpiòs', 'Ella F.',
        'AdStr']]
accoppiamenti=pd.DataFrame(data,columns=["girone","partita","squadra","capitano",
                                         "riserva","match 1",
                                         "match 2"]).set_index(["girone","partita"])
accoppiamenti



squadra        capitano  \
girone partita                                                  
A      1                          D Euforiche            Kuno   
       1                           Gli ultimi        crazycat   
B      1                          Green House     Bango Skank   
       1        Gli scrittori del quartierino         Marty12   
       2                           I Bananari  simone volponi   
       2                       the 3 BrOthers      AryaSophia   
C      1                        Living Stones          Macleo   
       1                          Le TreMende             Emy   
       2                   Retroguardia Trash        Andrea28   
       2                   Armata Brancaleone           AdStr   

                         riserva         match 1     match 2  
girone partita                                                
A      1                Marcello            Kuno      queffe  
       1                   Ghigo      caipiroska    crazycat  
B      1             Bango Skank           Plata      Joyopi  
       1              Federico72       camparino     Marty12  
       2              massimopud  simone volponi        Niko  
       2                ettore70             ITG  AryaSophia  
C      1        Vincenzo Iennaco          Ljuset      Macleo  
       1                  sefora            Thea         Emy  
       2               AlexComan        Andrea28       Eudes  
       2                Skorpiòs         Ella F.       AdStr

In [10]:
# Codice html per il post di riepilogo sugli accoppiamenti

prev_girone=""
for girone,partita in accoppiamenti.index.unique():
    if prev_girone!=girone:
        print("<p>&nbsp;</p>")
        print("<p><strong>Girone %s</strong></p>" % girone)
        prev_girone=girone
    p1=accoppiamenti.loc[girone,partita].iloc[0]
    p2=accoppiamenti.loc[girone,partita].iloc[1]
    print("<p>%d. %s - %s</p>" % (partita,p1["squadra"],p2["squadra"]) )
    print("<p style='margin-left: 40px;'>match 1: %s - %s</p>" % (p1["match 1"],p2["match 1"]))
    print("<p style='margin-left: 40px;'>match 2: %s - %s</p>" % (p1["match 2"],p2["match 2"]))
    

<p>&nbsp;</p>
<p><strong>Girone A</strong></p>
<p>1. D Euforiche - Gli ultimi</p>
<p style='margin-left: 40px;'>match 1: Kuno - caipiroska</p>
<p style='margin-left: 40px;'>match 2: queffe - crazycat</p>
<p>&nbsp;</p>
<p><strong>Girone B</strong></p>
<p>1. Green House - Gli scrittori del quartierino</p>
<p style='margin-left: 40px;'>match 1: Plata - camparino</p>
<p style='margin-left: 40px;'>match 2: Joyopi - Marty12</p>
<p>2. I Bananari - the 3 BrOthers</p>
<p style='margin-left: 40px;'>match 1: simone volponi - ITG</p>
<p style='margin-left: 40px;'>match 2: Niko - AryaSophia</p>
<p>&nbsp;</p>
<p><strong>Girone C</strong></p>
<p>1. Living Stones - Le TreMende</p>
<p style='margin-left: 40px;'>match 1: Ljuset - Thea</p>
<p style='margin-left: 40px;'>match 2: Macleo - Emy</p>
<p>2. Retroguardia Trash - Armata Brancaleone</p>
<p style='margin-left: 40px;'>match 1: Andrea28 - Ella F.</p>
<p style='margin-left: 40px;'>match 2: Eudes - AdStr</p>


## Fase 2. Creazione del tabellone per voti e controlli

In [11]:
importlib.reload(ms)
s=ms.N2017Spreadsheet()

# togliere quando c'è la versione definitiva delle formazioni
old_data=s.get_giornata(GIORNATA)
#old_data=None
s.set_giornata(GIORNATA,accoppiamenti,gironi,old_data)
#old_data

### Tabella ammissioni

In [12]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
ammissioni=s.get_giornata(GIORNATA)
ammissioni=ammissioni[["base","verifiche"]]

old_cols=ammissioni.columns.tolist()
new_cols=[c[1] for c in old_cols]
new_cols

ammissioni.columns=new_cols
a=accoppiamenti.reset_index().set_index("match 1")[["girone","partita","squadra","capitano","riserva"]]
a["match"]=1
a
b=accoppiamenti.reset_index().set_index("match 2")[["girone","partita","squadra","capitano","riserva"]]
b["match"]=2

ammissioni=ammissioni.merge(a.append(b),left_index=True,right_index=True)
new_index=["girone","partita","match","squadra"]
new_columns=["index","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]

ammissioni=ammissioni.reset_index().set_index(new_index).sort_index()[new_columns]
new_columns=["titolare","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]
ammissioni.columns=new_columns

In [13]:
def calc_ammissioni(row):
    row["sostituzione"]=""
    if row["ingresso riserva"]:
        sost="("+row["riserva"]+" da "+row["ingresso riserva"]+")"
        row["sostituzione"]=sost
    row["punti penalità"]=""
    if int(row["penalità"])==0: return row
    if abs(int(row["penalità"]))==1:
        row["punti penalità"]="1 punto di penalità"
    else:
        row["punti penalità"]="%d punti di penalità" % abs(int(row["penalità"]))
    return row

ammissioni=ammissioni.apply(calc_ammissioni,axis=1)
ammissioni

titolare  \
girone partita match squadra                                         
A      1       1     D Euforiche                              Kuno   
                     Gli ultimi                         caipiroska   
               2     D Euforiche                            queffe   
                     Gli ultimi                           crazycat   
B      1       1     Gli scrittori del quartierino       camparino   
                     Green House                             Plata   
               2     Gli scrittori del quartierino         Marty12   
                     Green House                            Joyopi   
       2       1     I Bananari                     simone volponi   
                     the 3 BrOthers                            ITG   
               2     I Bananari                               Niko   
                     the 3 BrOthers                     AryaSophia   
C      1       1     Le TreMende                              Thea   
                     Living Stones                          Ljuset   
               2     Le TreMende                               Emy   
                     Living Stones                          Macleo   
       2       1     Armata Brancaleone                    Ella F.   
                     Retroguardia Trash                   Andrea28   
               2     Armata Brancaleone                      AdStr   
                     Retroguardia Trash                      Eudes   

                                                             riserva  \
girone partita match squadra                                           
A      1       1     D Euforiche                            Marcello   
                     Gli ultimi                                Ghigo   
               2     D Euforiche                            Marcello   
                     Gli ultimi                                Ghigo   
B      1       1     Gli scrittori del quartierino        Federico72   
                     Green House                         Bango Skank   
               2     Gli scrittori del quartierino        Federico72   
                     Green House                         Bango Skank   
       2       1     I Bananari                           massimopud   
                     the 3 BrOthers                         ettore70   
               2     I Bananari                           massimopud   
                     the 3 BrOthers                         ettore70   
C      1       1     Le TreMende                              sefora   
                     Living Stones                  Vincenzo Iennaco   
               2     Le TreMende                              sefora   
                     Living Stones                  Vincenzo Iennaco   
       2       1     Armata Brancaleone                     Skorpiòs   
                     Retroguardia Trash                    AlexComan   
               2     Armata Brancaleone                     Skorpiòs   
                     Retroguardia Trash                    AlexComan   

                                                   ingresso riserva  \
girone partita match squadra                                          
A      1       1     D Euforiche                                      
                     Gli ultimi                                       
               2     D Euforiche                                      
                     Gli ultimi                                       
B      1       1     Gli scrittori del quartierino                    
                     Green House                                      
               2     Gli scrittori del quartierino                    
                     Green House                                      
       2       1     I Bananari                                       
                     the 3 BrOthers                                   
               2     I Bananari                                       
         

In [14]:

#old_cols=ammissioni.columns.tolist()
#new_cols=[c[1] for c in old_cols]
#new_cols

#ammissioni.columns=new_cols

mu.to_table_html_multi(ammissioni[["titolare","titolo",
                                   "lunghezza"]])

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>match</th>
<th>squadra</th>
<th>titolare</th>
<th>titolo</th>
<th>lunghezza</th>
</tr>
<tr>
<th rowspan='4'>A</th>
<th rowspan='8'>1</th>
<th rowspan='2'>1</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>Kuno</td>
<td style='text-align:center;'>Il fuoco è arancione, rosso e giallo</td>
<td style='text-align:center;'>7923</td>
</tr>
<tr>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>caipiroska</td>
<td style='text-align:center;'>I falò si Silver Lake</td>
<td style='text-align:center;'>5158</td>
</tr>
<tr>
<th rowspan='2'>2</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>queffe</td>
<td style='text-align:center;'>Il grande falò</td>
<td style='text-align:center;'>7245</td>
</tr>
<tr>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>crazycat</td>
<td style='text-align:center;'>Riscaldando il cielo</td>
<td style='text-align:cent

## Fase 3. Recupero dati

In [15]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
voti=s.get_giornata(GIORNATA)


In [16]:
### Simulazione di riempimento del tabellone - da togliere quando ci sono i voti definitivi
importlib.reload(mu)

#voti["base","titolo"]=voti["base","titolo"].apply(mu.random_title)
#voti["base","ingresso riserva"]=voti["base","ingresso riserva"].apply(mu.random_riserva)
#voti["verifiche","lunghezza"]=voti["verifiche","lunghezza"].apply(mu.random_len)
#voti["verifiche","link"]=voti["verifiche","link"].apply(mu.random_verifica)
#voti["verifiche","link commento"]=voti["verifiche","link commento"].apply(mu.random_verifica)
#voti["verifiche","tag"]=voti["verifiche","tag"].apply(mu.random_verifica)
#voti["verifiche","commento valido"]=voti["verifiche","commento valido"].apply(mu.random_verifica)

#for G,Gpart_list,altri in [ ("A",[1],[("B",1),("B",2),("C",1),("C",2)]),
#                 ("B",[1,2],[("A",1),("C",1),("C",2)]),
#                 ("C",[1,2],[("A",1),("B",1),("B",2)]) ]:
#    for sq in gironi[gironi[G]!="(riposo)"][G]:
#        for part in Gpart_list:
#            for m in ["match 1","match 2"]:
#                giocatore_1=accoppiamenti.loc[G,part][m].iloc[0]
#                giocatore_2=accoppiamenti.loc[G,part][m].iloc[1]
#                goal_1=0
#                goal_2=0
#                voti.set_value(giocatore_1,(G,sq),goal_1)
#                voti.set_value(giocatore_2,(G,sq),goal_2)
#            
#        for gir,part in altri:
#            for m in ["match 1","match 2"]:
#                giocatore_1=accoppiamenti.loc[gir,part][m].iloc[0]
#                giocatore_2=accoppiamenti.loc[gir,part][m].iloc[1]
#                goal_1=random.randint(0,2)
#                goal_2=2-goal_1
#                voti.set_value(giocatore_1,(G,sq),goal_1)
#                voti.set_value(giocatore_2,(G,sq),goal_2)

#voti["verifiche","penalità"]=voti["verifiche","lunghezza"].apply(mu.calc_penalita_lungh)
#for k in ["tag","link","link commento","commento valido"]:
#    voti["verifiche","penalità"]+=voti["verifiche",k].apply(mu.calc_penalita)


<module 'mylib.utility' from '/home/chiara/casa/wd/scrittopoli/mylib/utility.py'>

In [17]:
voti["verifiche","penalità"]=pd.to_numeric(voti["verifiche","penalità"],downcast="signed")
voti["base","goal giocatore"]=voti[["A","B","C"]].sum(axis=1,level=1).sum(axis=1)+voti["verifiche","penalità"]
voti["base","goal giocatore"]=pd.to_numeric(voti["base","goal giocatore"],downcast="signed")
voti

base  \
               ingresso riserva   
titolare                          
Kuno                              
queffe                            
caipiroska                        
crazycat                          
Plata                             
Joyopi                            
camparino                         
Marty12                           
simone volponi                    
Niko                              
ITG                               
AryaSophia                        
Ljuset                            
Macleo                            
Thea                              
Emy                               
Andrea28                          
Eudes                             
Ella F.                           
AdStr                             

                                                              verifiche      \
                                                       titolo lunghezza tag   
titolare                                                                      
Kuno                     Il fuoco è arancione, rosso e giallo      7923   X   
queffe                                         Il grande falò      7245   X   
caipiroska                              I falò si Silver Lake      5158   X   
crazycat                                 Riscaldando il cielo      7520   X   
Plata                                     Johnny the dead man      6370   X   
Joyopi                            Una storia che fa incazzare      5916   X   
camparino                         Ultimo tango sulla spiaggia      2449   X   
Marty12                                   Gli amanti del falò      7989   X   
simone volponi                                Black Rock City      7962   X   
Niko            Odore d'erba e di fiori, di tenerezza e amore      7983   X   
ITG                                        Le forme dei barrè      7347   X   
AryaSophia                               Fantasma del passato      6080   X   
Ljuset                                      Fuochi sulla neve      8000   X   
Macleo                                 Li Flama de San Lorenz      5483   X   
Thea                                         Il fuoco perenne      7658   X   
Emy                                     Una donna senza volto      7920   X   
Andrea28         Il Diavolo, Mario Balotelli e Raffaella Fico      7974       
Eudes                                     L'immensa discarica      7999   X   
Ella F.                                               Catarsi      5356   X   
AdStr                                             Lo squarcio      5027   X   

                                                                     A  \
               link link commento commento valido penalità D Euforiche   
titolare                                                                 
Kuno              X             X               X        0           0   
queffe            X             X               X        0           0   
caipiroska        X             X               X        0           0   
crazycat          X             X               X        0           0   
Plata             X             X               X        0           1   
Joyopi            X             X               X        0           2   
camparino         X             X               X        0           1   
Marty12           X             X               X        0           0   
simone volponi    X             X               X        0           0   
Niko              X             X               X        0           2   
ITG               X             X               X        0           2   
AryaSophia        X             X               X        0           0   
Ljuset            X             X               X        0           1   
Macleo            X             X               X        0           2   
Thea              X             X               X        0           1   
Emy               X             X               X        0           0 

## Calcolo dei risultati

### Risultati della seconda giornata

In [18]:
risultati=accoppiamenti.copy()

risultati=risultati.reset_index().set_index("squadra")
risultati["match 1 goal"]=0
risultati["match 1 riserva"]=False
risultati["match 2 goal"]=0
risultati["match 2 riserva"]=False
risultati["goal"]=0
risultati["penalità"]=0


for sq in risultati.index:
    m1=risultati.loc[sq]["match 1"]
    m2=risultati.loc[sq]["match 2"]
    goal_1=voti.loc[m1]["base","goal giocatore"]
    goal_2=voti.loc[m2]["base","goal giocatore"]
    pen_1=voti.loc[m1]["verifiche","penalità"]
    pen_2=voti.loc[m2]["verifiche","penalità"]
    risultati.set_value(sq,"match 1 goal",goal_1)
    risultati.set_value(sq,"match 2 goal",goal_2)
    risultati.set_value(sq,"goal",goal_1+goal_2)
    risultati.set_value(sq,"penalità",pen_1+pen_2)
    ris_1=bool(voti.loc[m1]["base","ingresso riserva"])
    risultati.set_value(sq,"match 1 riserva",ris_1)
    if ris_1: continue
    risultati.set_value(sq,"match 2 riserva",bool(voti.loc[m2]["base","ingresso riserva"]))
    
risultati=risultati.reset_index().set_index(["girone","partita"])
sc_dati=[]
for girone,partita in risultati.index.unique():
    squadra_1=risultati.loc[girone,partita]["squadra"].iloc[0]
    squadra_2=risultati.loc[girone,partita]["squadra"].iloc[1]
    goal_1=risultati.loc[girone,partita]["goal"].iloc[0]
    goal_2=risultati.loc[girone,partita]["goal"].iloc[1]
    pen_1=risultati.loc[girone,partita]["penalità"].iloc[0]
    pen_2=risultati.loc[girone,partita]["penalità"].iloc[1]
    m1_goal_1=risultati.loc[girone,partita]["match 1 goal"].iloc[0]
    m1_goal_2=risultati.loc[girone,partita]["match 1 goal"].iloc[1]
    m2_goal_1=risultati.loc[girone,partita]["match 2 goal"].iloc[0]
    m2_goal_2=risultati.loc[girone,partita]["match 2 goal"].iloc[1]
    sc_dati.append([girone,partita,squadra_1,squadra_2,goal_1,goal_2,
                    m1_goal_1,m1_goal_2,m2_goal_1,m2_goal_2,pen_1,pen_2])

risultati=risultati.reset_index().set_index(["girone","partita","squadra"])

schedina=pd.DataFrame(sc_dati,columns=["girone","partita","squadra 1","squadra 2",
                                       "goal squadra 1","goal squadra 2",
                                       "match 1 squadra 1","match 1 squadra 2",
                                       "match 2 squadra 1","match 2 squadra 2",
                                       "penalità squadra 1","penalità squadra 2"])

out_schedina=schedina.copy()
out_schedina["titolo"]=out_schedina["squadra 1"]+" - "+out_schedina["squadra 2"]
out_schedina["risultato"]=( out_schedina["goal squadra 1"].apply(str)
                            +" - "+out_schedina["goal squadra 2"].apply(str) )
out_schedina["match 1"]=( out_schedina["match 1 squadra 1"].apply(str)
                          +" - "+out_schedina["match 1 squadra 2"].apply(str) )
out_schedina["match 2"]=( out_schedina["match 2 squadra 1"].apply(str)
                          +" - "+out_schedina["match 2 squadra 2"].apply(str) )
out_schedina=out_schedina.set_index(["girone","partita"])

out_schedina=out_schedina[["titolo","match 1","match 2","risultato"]]


risultati

capitano  \
girone partita squadra                                         
A      1       D Euforiche                              Kuno   
               Gli ultimi                           crazycat   
B      1       Green House                       Bango Skank   
               Gli scrittori del quartierino         Marty12   
       2       I Bananari                     simone volponi   
               the 3 BrOthers                     AryaSophia   
C      1       Living Stones                          Macleo   
               Le TreMende                               Emy   
       2       Retroguardia Trash                   Andrea28   
               Armata Brancaleone                      AdStr   

                                                       riserva  \
girone partita squadra                                           
A      1       D Euforiche                            Marcello   
               Gli ultimi                                Ghigo   
B      1       Green House                         Bango Skank   
               Gli scrittori del quartierino        Federico72   
       2       I Bananari                           massimopud   
               the 3 BrOthers                         ettore70   
C      1       Living Stones                  Vincenzo Iennaco   
               Le TreMende                              sefora   
       2       Retroguardia Trash                    AlexComan   
               Armata Brancaleone                     Skorpiòs   

                                                     match 1     match 2  \
girone partita squadra                                                     
A      1       D Euforiche                              Kuno      queffe   
               Gli ultimi                         caipiroska    crazycat   
B      1       Green House                             Plata      Joyopi   
               Gli scrittori del quartierino       camparino     Marty12   
       2       I Bananari                     simone volponi        Niko   
               the 3 BrOthers                            ITG  AryaSophia   
C      1       Living Stones                          Ljuset      Macleo   
               Le TreMende                              Thea         Emy   
       2       Retroguardia Trash                   Andrea28       Eudes   
               Armata Brancaleone                    Ella F.       AdStr   

                                              match 1 goal  match 1 riserva  \
girone partita squadra                                                        
A      1       D Euforiche                              15            False   
               Gli ultimi                                1            False   
B      1       Green House                               7            False   
               Gli scrittori del quartierino             7            False   
       2       I Bananari                                1            False   
               the 3 BrOthers                           13            False   
C      1       Living Stones                             8            False   
               Le TreMende                               6            False   
       2       Retroguardia Trash                        1            False   
               Armata Brancaleone                       12            False   

                                              match 2 goal  match 2 riserva  \
girone partita squadra                                                        
A      1       D Euforiche                               9            False   
               Gli ultimi                                7            False   
B      1       Green House                              14            False   
               Gli scrittori del quartierino             0            False   
       2       I Bananari                               14            False   
               the 3 BrOthers                            0            False

In [19]:
schedina

,girone,partita,squadra 1,squadra 2,goal squadra 1,goal squadra 2,match 1 squadra 1,match 1 squadra 2,match 2 squadra 1,match 2 squadra 2,penalità squadra 1,penalità squadra 2
0,A,1,D Euforiche,Gli ultimi,24,8,15,1,9,7,0,0
1,B,1,Green House,Gli scrittori del quartierino,21,7,7,7,14,0,0,0
2,B,2,I Bananari,the 3 BrOthers,15,13,1,13,14,0,0,0
3,C,1,Living Stones,Le TreMende,14,14,8,6,6,8,0,0
4,C,2,Retroguardia Trash,Armata Brancaleone,10,17,1,12,9,5,-1,0


In [20]:
out_schedina

titolo match 1 match 2  \
girone partita                                                                
A      1                           D Euforiche - Gli ultimi  15 - 1   9 - 7   
B      1        Green House - Gli scrittori del quartierino   7 - 7  14 - 0   
       2                        I Bananari - the 3 BrOthers  1 - 13  14 - 0   
C      1                        Living Stones - Le TreMende   8 - 6   6 - 8   
       2            Retroguardia Trash - Armata Brancaleone  1 - 12   9 - 5   

               risultato  
girone partita            
A      1          24 - 8  
B      1          21 - 7  
       2         15 - 13  
C      1         14 - 14  
       2         10 - 17

In [21]:
# output per il post dei risultati

mu.to_table_html_multi(out_schedina)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>titolo</th>
<th>match 1</th>
<th>match 2</th>
<th>risultato</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>D Euforiche - Gli ultimi</td>
<td style='text-align:center;'>15 - 1</td>
<td style='text-align:center;'>9 - 7</td>
<td style='text-align:center;'>24 - 8</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House - Gli scrittori del quartierino</td>
<td style='text-align:center;'>7 - 7</td>
<td style='text-align:center;'>14 - 0</td>
<td style='text-align:center;'>21 - 7</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>I Bananari - the 3 BrOthers</td>
<td style='text-align:center;'>1 - 13</td>
<td style='text-align:center;'>14 - 0</td>
<td style='text-align:center;'>15 - 13</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones - Le TreMende</td>
<td style

### Classifica


Ordine dei criteri:
* classifiche avulse (scontri diretti);
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti nell'ultimo incontro;
* numero di reti segnate nell'ultimo incontro;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.

Nella prima giornata non ci sono classifiche avulse e l'unico incontro è anche l'ultimo, quindi si considerano solo:
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.


In [22]:
## Tabella degli scontri diretti

sg1=schedina_g1.copy()
sg2=schedina.copy()

diretti=sg1[["girone","squadra 1","squadra 2","goal squadra 1","goal squadra 2"]].copy()
diretti=diretti.append(sg2[["girone","squadra 1","squadra 2","goal squadra 1","goal squadra 2"]])
d=diretti.copy()
diretti.columns=["girone","squadra","avversario","fatti","subiti"]
d.columns=["girone","avversario","squadra","subiti","fatti"]

diretti=diretti.append(d).set_index(["girone","squadra","avversario"])[["fatti","subiti"]].sort_index()
diretti

fatti  \
girone squadra                       avversario                             
A      D Euforiche                   Gli ultimi                        24   
                                     La threesome                      21   
       Gli ultimi                    D Euforiche                        8   
       La threesome                  D Euforiche                       11   
B      Gli scrittori del quartierino Green House                        7   
                                     the 3 BrOthers                    16   
       Green House                   Gli scrittori del quartierino     21   
                                     I Bananari                        12   
       I Bananari                    Green House                       16   
                                     the 3 BrOthers                    15   
       the 3 BrOthers                Gli scrittori del quartierino     12   
                                     I Bananari                        13   
C      Armata Brancaleone            Le TreMende                       18   
                                     Retroguardia Trash                17   
       Le TreMende                   Armata Brancaleone                10   
                                     Living Stones                     14   
       Living Stones                 Le TreMende                       14   
                                     Retroguardia Trash                 7   
       Retroguardia Trash            Armata Brancaleone                10   
                                     Living Stones                     21   

                                                                    subiti  
girone squadra                       avversario                             
A      D Euforiche                   Gli ultimi                          8  
                                     La threesome                       11  
       Gli ultimi                    D Euforiche                        24  
       La threesome                  D Euforiche                        21  
B      Gli scrittori del quartierino Green House                        21  
                                     the 3 BrOthers                     12  
       Green House                   Gli scrittori del quartierino       7  
                                     I Bananari                         16  
       I Bananari                    Green House                        12  
                                     the 3 BrOthers                     13  
       the 3 BrOthers                Gli scrittori del quartierino      16  
                                     I Bananari                         15  
C      Armata Brancaleone            Le TreMende                        10  
                                     Retroguardia Trash                 10  
       Le TreMende                   Armata Brancaleone                 18  
                                     Living Stones                      14  
       Living Stones                 Le TreMende                        14  
                                     Retroguardia Trash                 21  
       Retroguardia Trash            Armata Brancaleone                 17  
                                     Living Stones                       7

In [23]:
def calc_vittorie(row):
    row["vittorie"]=0
    row["sconfitte"]=0
    row["pareggi"]=0
    if row["goal fatti"]>row["goal subiti"]:
        row["vittorie"]=1
    elif row["goal fatti"]<row["goal subiti"]:
        row["sconfitte"]=1
    else:
        row["pareggi"]=1
    #row["punti"]=3*row["vittorie"]+row["pareggi"]
    return row

q=schedina.copy()

a=q[["girone","squadra 1","goal squadra 1","goal squadra 2",
     "penalità squadra 1","penalità squadra 2"]].copy()
b=q[["girone","squadra 2","goal squadra 2","goal squadra 1",
     "penalità squadra 2","penalità squadra 1"]].copy()

a.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]
b.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]

classifica_g2=a.append(b).set_index(["girone","squadra"]).sort_index()
classifica_g2=classifica_g2.apply(calc_vittorie,axis=1)

df=pd.DataFrame([["A",riposano.iloc[0]["squadra 1"],0]],
                columns=["girone","squadra","goal fatti"])
df=df.set_index(["girone","squadra"])
classifica_g2=classifica_g2.append(df).fillna(0).sort_index()


classifica_g2["goal fatti ultimo incontro"]=classifica_g2["goal fatti"]
classifica_g2["goal subiti ultimo incontro"]=classifica_g2["goal subiti"]
classifica_g1["goal fatti ultimo incontro"]=0
classifica_g1["goal subiti ultimo incontro"]=0

base_fields=["goal fatti","goal subiti","penalità fatte",
             "penalità subite","vittorie","sconfitte","pareggi",
             "goal fatti ultimo incontro","goal subiti ultimo incontro"]

classifica_g2.merge(classifica_g1[base_fields],left_index=True,right_index=True,
                    how="outer",suffixes=[" g1"," g2"]).fillna(0)

classifica=classifica_g2+classifica_g1[base_fields]
classifica["goal netti fatti"]=classifica["goal fatti"]-classifica["penalità fatte"]
classifica["goal netti subiti"]=classifica["goal subiti"]-classifica["penalità subite"]

classifica["differenza reti"]=classifica["goal fatti"]-classifica["goal subiti"]
classifica["differenza reti netta"]=classifica["goal netti fatti"]-classifica["goal netti subiti"]
classifica["differenza reti ultimo incontro"]=classifica["goal fatti ultimo incontro"]-classifica["goal subiti ultimo incontro"]

classifica["punti"]=3*classifica["vittorie"]+classifica["pareggi"]

#df=pd.DataFrame([["A","Gli ultimi",0]],columns=["girone","squadra","goal fatti"])
#df=df.set_index(["girone","squadra"])
#classifica=classifica.append(df).fillna(0).sort_index()
classifica=classifica[["punti","vittorie","pareggi","sconfitte",
                       "differenza reti",
                       "goal fatti","goal subiti",
                       "differenza reti netta",
                       "goal netti fatti","goal netti subiti",
                       "penalità fatte","penalità subite",
                       "differenza reti ultimo incontro",
                       "goal fatti ultimo incontro","goal subiti ultimo incontro"]]
classifica=classifica.astype(int)

In [24]:
importlib.reload(mu)

classifica=mu.sort_classifica(classifica.reset_index(),diretti)
classifica=classifica.set_index(["girone","squadra"])
classifica



punti  vittorie  pareggi  sconfitte  \
girone squadra                                                              
A      D Euforiche                        6         2        0          0   
       La threesome                       0         0        0          1   
       Gli ultimi                         0         0        0          1   
B      I Bananari                         6         2        0          0   
       Green House                        3         1        0          1   
       Gli scrittori del quartierino      3         1        0          1   
       the 3 BrOthers                     0         0        0          2   
C      Armata Brancaleone                 6         2        0          0   
       Retroguardia Trash                 3         1        0          1   
       Le TreMende                        1         0        1          1   
       Living Stones                      1         0        1          1   

                                      differenza reti  goal fatti  \
girone squadra                                                      
A      D Euforiche                                 26          45   
       La threesome                               -10          11   
       Gli ultimi                                 -16           8   
B      I Bananari                                   6          31   
       Green House                                 10          33   
       Gli scrittori del quartierino              -10          23   
       the 3 BrOthers                              -6          25   
C      Armata Brancaleone                          15          35   
       Retroguardia Trash                           7          31   
       Le TreMende                                 -8          24   
       Living Stones                              -14          21   

                                      goal subiti  differenza reti netta  \
girone squadra                                                             
A      D Euforiche                             19                     26   
       La threesome                            21                    -10   
       Gli ultimi                              24                    -16   
B      I Bananari                              25                      6   
       Green House                             23                     10   
       Gli scrittori del quartierino           33                    -10   
       the 3 BrOthers                          31                     -6   
C      Armata Brancaleone                      20                     14   
       Retroguardia Trash                      24                      8   
       Le TreMende                             32                     -8   
       Living Stones                           35                    -14   

                                      goal netti fatti  goal netti subiti  \
girone squadra                                                              
A      D Euforiche                                  45                 19   
       La threesome                                 11                 21   
       Gli ultimi                                    8                 24   
B      I Bananari                                   31                 25   
       Green House                                  33                 23   
       Gli scrittori del quartierino                23                 33   
       the 3 BrOthers                               25                 31   
C      Armata Brancaleone                           35                 21   
       Retroguardia Trash                           32                 24   
       Le TreMende                                  24                 32   
       Living Stones                                21                 35   

                                      penalità fatte  penalità subite  \
girone squadra                                                          
A

In [25]:
diretti

fatti  \
girone squadra                       avversario                             
A      D Euforiche                   Gli ultimi                        24   
                                     La threesome                      21   
       Gli ultimi                    D Euforiche                        8   
       La threesome                  D Euforiche                       11   
B      Gli scrittori del quartierino Green House                        7   
                                     the 3 BrOthers                    16   
       Green House                   Gli scrittori del quartierino     21   
                                     I Bananari                        12   
       I Bananari                    Green House                       16   
                                     the 3 BrOthers                    15   
       the 3 BrOthers                Gli scrittori del quartierino     12   
                                     I Bananari                        13   
C      Armata Brancaleone            Le TreMende                       18   
                                     Retroguardia Trash                17   
       Le TreMende                   Armata Brancaleone                10   
                                     Living Stones                     14   
       Living Stones                 Le TreMende                       14   
                                     Retroguardia Trash                 7   
       Retroguardia Trash            Armata Brancaleone                10   
                                     Living Stones                     21   

                                                                    subiti  
girone squadra                       avversario                             
A      D Euforiche                   Gli ultimi                          8  
                                     La threesome                       11  
       Gli ultimi                    D Euforiche                        24  
       La threesome                  D Euforiche                        21  
B      Gli scrittori del quartierino Green House                        21  
                                     the 3 BrOthers                     12  
       Green House                   Gli scrittori del quartierino       7  
                                     I Bananari                         16  
       I Bananari                    Green House                        12  
                                     the 3 BrOthers                     13  
       the 3 BrOthers                Gli scrittori del quartierino      16  
                                     I Bananari                         15  
C      Armata Brancaleone            Le TreMende                        10  
                                     Retroguardia Trash                 10  
       Le TreMende                   Armata Brancaleone                 18  
                                     Living Stones                      14  
       Living Stones                 Le TreMende                        14  
                                     Retroguardia Trash                 21  
       Retroguardia Trash            Armata Brancaleone                 17  
                                     Living Stones                       7

In [26]:
## Output per il post della classifica

mu.to_table_html_multi(classifica)

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>squadra</th>
<th>punti</th>
<th>vittorie</th>
<th>pareggi</th>
<th>sconfitte</th>
<th>differenza reti</th>
<th>goal fatti</th>
<th>goal subiti</th>
<th>differenza reti netta</th>
<th>goal netti fatti</th>
<th>goal netti subiti</th>
<th>penalità fatte</th>
<th>penalità subite</th>
<th>differenza reti ultimo incontro</th>
<th>goal fatti ultimo incontro</th>
<th>goal subiti ultimo incontro</th>
</tr>
<tr>
<th rowspan='3'>A</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>6</td>
<td style='text-align:center;'>2</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>26</td>
<td style='text-align:center;'>45</td>
<td style='text-align:center;'>19</td>
<td style='text-align:center;'>26</td>
<td style='text-align:center;'>45</td>
<td style='text-align:center;'>19</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td>
<td 

In [27]:
# diretti
mu.to_table_html_multi(diretti)

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>squadra</th>
<th>avversario</th>
<th>fatti</th>
<th>subiti</th>
</tr>
<tr>
<th rowspan='4'>A</th>
<th rowspan='2'>D Euforiche</th>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>24</td>
<td style='text-align:center;'>8</td>
</tr>
<tr>
<th rowspan='1'>La threesome</th>
<td style='text-align:center;'>21</td>
<td style='text-align:center;'>11</td>
</tr>
<tr>
<th rowspan='1'>Gli ultimi</th>
<th rowspan='2'>D Euforiche</th>
<td style='text-align:center;'>8</td>
<td style='text-align:center;'>24</td>
</tr>
<tr>
<th rowspan='1'>La threesome</th>
<td style='text-align:center;'>11</td>
<td style='text-align:center;'>21</td>
</tr>
<tr>
<th rowspan='8'>B</th>
<th rowspan='2'>Gli scrittori del quartierino</th>
<th rowspan='1'>Green House</th>
<td style='text-align:center;'>7</td>
<td style='text-align:center;'>21</td>
</tr>
<tr>
<th rowspan='1'>the 3 BrOthers</th>
<td style='text-align:center;'>16</td>
<td styl

### Nuovi punteggi Elo

In [28]:
q=risultati.copy()

def calc_giocatore(row,match):
    riserva=row["riserva"]
    giocatore=row["match %d" % match]
    sostituzione=row["match %d riserva" % match]
    if sostituzione: return riserva
    return giocatore

def calc_giocatore_1(row): return calc_giocatore(row,1)
def calc_giocatore_2(row): return calc_giocatore(row,2)

q["giocatore 1"]=q.apply(calc_giocatore_1,axis=1)
q["giocatore 2"]=q.apply(calc_giocatore_2,axis=1)

a=q[["giocatore 1","match 1 goal"]].copy()
a.columns=["giocatore","goal"]
a["match"]="1"

b=q[["giocatore 2","match 2 goal"]].copy()
b.columns=["giocatore","goal"]
b["match"]="2"

q=a.append(b).reset_index().set_index(["girone","partita","match"])
q=q.merge(q,how="outer",left_index=True,right_index=True,suffixes=[" 1"," 2"])
q=q[q["giocatore 1"]!=q["giocatore 2"]]

elo=q[["squadra 1","giocatore 1","giocatore 2","goal 1","goal 2"]]
elo.columns=["squadra","giocatore","avversario","goal fatti","goal subiti"]
elo=elo.reset_index().set_index(["girone","squadra","giocatore"]).sort_index()
elo=elo[["avversario","goal fatti","goal subiti"]].copy()
#elo["elo precedente"]=1500
#elo["elo avversario"]=1500

base_elo=elo_g_g1[["elo"]].copy()

base_elo.columns=["elo precedente"]
elo=elo.reset_index().set_index("giocatore").merge(base_elo,left_index=True,right_index=True)
base_elo.columns=["elo avversario"]
elo=elo.reset_index().set_index("avversario").merge(base_elo,left_index=True,right_index=True)
elo=elo.reset_index().set_index(["girone","squadra","giocatore"])
elo.columns=["avversario","goal fatti","goal subiti","elo precedente","elo avversario"]

def calcola_punteggi(row):
    exp_a,exp_b=melo.punteggi_attesi(row["elo precedente"],row["elo avversario"])
    score_a,score_b=melo.punteggi_reali(row["goal fatti"],row["goal subiti"])
    row["punteggio atteso"]=exp_a
    row["punteggio reale"]=score_a
    return row

elo=elo.apply(calcola_punteggi,axis=1)

elo["scostamento"]=elo["punteggio reale"]-elo["punteggio atteso"]
elo=elo.reset_index().set_index(["giocatore"])
elo

,girone,squadra,avversario,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento
giocatore,,,,,,,,,,
Kuno,A,D Euforiche,caipiroska,15,1,1525,1500,0.535916,0.937500,0.401584
queffe,A,D Euforiche,crazycat,9,7,1505,1500,0.507195,0.562500,0.055305
caipiroska,A,Gli ultimi,Kuno,1,15,1500,1525,0.464084,0.062500,-0.401584
crazycat,A,Gli ultimi,queffe,7,9,1500,1505,0.492805,0.437500,-0.055305
Marty12,B,Gli scrittori del quartierino,Joyopi,0,14,1478,1488,0.485613,0.000000,-0.485613
camparino,B,Gli scrittori del quartierino,Plata,7,7,1502,1498,0.505756,0.500000,-0.005756
Joyopi,B,Green House,Marty12,14,0,1488,1478,0.514387,1.000000,0.485613
Plata,B,Green House,camparino,7,7,1498,1502,0.494244,0.500000,0.005756
Niko,B,I Bananari,AryaSophia,14,0,1502,1465,0.553047,1.000000,0.446953


In [29]:
elo_s_prec=elo_g_g1.reset_index().set_index("squadra").sort_index().groupby(level=0).mean().copy()

a=schedina.copy()[["girone","squadra 1","squadra 2","goal squadra 1","goal squadra 2"]]
b=schedina.copy()[["girone","squadra 2","squadra 1","goal squadra 2","goal squadra 1"]]
a.columns=["girone","squadra","avversario","goal fatti","goal subiti"]
b.columns=["girone","squadra","avversario","goal fatti","goal subiti"]

elo_s=a.append(b).set_index("squadra")

elo_s_prec.columns=["elo precedente"]
elo_s=elo_s.merge(elo_s_prec,left_index=True,right_index=True)
elo_s_prec.columns=["elo avversario"]
elo_s=elo_s.reset_index().set_index("avversario").merge(elo_s_prec,left_index=True,
                                                        right_index=True)

elo_s=elo_s.reset_index().set_index("squadra")
elo_s.columns=["avversario","girone","goal fatti","goal subiti","elo precedente","elo avversario"]
elo_s=elo_s.apply(calcola_punteggi,axis=1)
elo_s["scostamento squadra"]=elo_s["punteggio reale"]-elo_s["punteggio atteso"]

rip_sq=riposano.iloc[0]["squadra 1"]
rip_mean=elo_g_g1[elo_g_g1["squadra"]==rip_sq]["elo"].mean()

rip=pd.DataFrame([[rip_sq,"-","A",0,0,rip_mean,rip_mean,0.5,0.5,0.0]],
                 columns=["squadra"]+list(elo_s.columns))

elo_s=elo_s.append(rip.set_index("squadra"))
elo_s

,avversario,girone,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento squadra
squadra,,,,,,,,,
D Euforiche,Gli ultimi,A,24,8,1511.666667,1500.000000,0.516783,0.750000,0.233217
Green House,Gli scrittori del quartierino,B,21,7,1494.666667,1505.000000,0.485134,0.750000,0.264866
I Bananari,the 3 BrOthers,B,15,13,1505.333333,1495.000000,0.514866,0.535714,0.020848
Living Stones,Le TreMende,C,14,14,1481.666667,1489.666667,0.488489,0.500000,0.011511
Retroguardia Trash,Armata Brancaleone,C,10,17,1518.333333,1510.333333,0.511511,0.370370,-0.141141
Gli ultimi,D Euforiche,A,8,24,1500.000000,1511.666667,0.483217,0.250000,-0.233217
Gli scrittori del quartierino,Green House,B,7,21,1505.000000,1494.666667,0.514866,0.250000,-0.264866
the 3 BrOthers,I Bananari,B,13,15,1495.000000,1505.333333,0.485134,0.464286,-0.020848
Le TreMende,Living Stones,C,14,14,1489.666667,1481.666667,0.511511,0.500000,-0.011511


In [30]:

q=elo_s.reset_index()[["girone","squadra","scostamento squadra"]]
#elo.merge(d,on="squadra")
q=squadre.reset_index().merge(q,on="squadra").set_index(["giocatore","squadra","girone"])
elo=elo.reset_index().set_index(["giocatore","squadra","girone"]).merge(q[["scostamento squadra"]],left_index=True,right_index=True,how="outer")
elo["goal fatti"]=pd.to_numeric(elo["goal fatti"].fillna(0),downcast="signed")
elo["goal subiti"]=pd.to_numeric(elo["goal subiti"].fillna(0),downcast="signed")
elo["elo precedente"]=pd.to_numeric(elo["elo precedente"].fillna(1500),downcast="signed")
elo["elo avversario"]=pd.to_numeric(elo["elo avversario"].fillna(1500),downcast="signed")
elo["punteggio atteso"]=elo["punteggio atteso"].fillna(0.5)
elo["punteggio reale"]=elo["punteggio reale"].fillna(0.5)
elo["scostamento"]=elo["scostamento"].fillna(0.0)
elo["avversario"]=elo["avversario"].fillna("(non giocato)")

elo=elo.reset_index().set_index(["giocatore"])

elo



,squadra,girone,avversario,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento,scostamento squadra
giocatore,,,,,,,,,,,
AdStr,Armata Brancaleone,C,Eudes,5,9,1504,1507,0.495683,0.357143,-0.138540,0.141141
AlexComan,Retroguardia Trash,C,(non giocato),0,0,1500,1500,0.500000,0.500000,0.000000,-0.141141
Andrea28,Retroguardia Trash,C,Ella F.,1,12,1526,1537,0.484175,0.076923,-0.407252,-0.141141
AryaSophia,the 3 BrOthers,B,Niko,0,14,1465,1502,0.446953,0.000000,-0.446953,-0.020848
Bango Skank,Green House,B,(non giocato),0,0,1500,1500,0.500000,0.500000,0.000000,0.264866
Ella F.,Armata Brancaleone,C,Andrea28,12,1,1537,1526,0.515825,0.923077,0.407252,0.141141
Emy,Le TreMende,C,Macleo,8,6,1463,1474,0.484175,0.571429,0.087254,-0.011511
Eudes,Retroguardia Trash,C,AdStr,9,5,1507,1504,0.504317,0.642857,0.138540,-0.141141
Federico72,Gli scrittori del quartierino,B,(non giocato),0,0,1500,1500,0.500000,0.500000,0.000000,-0.264866


In [31]:
def calc_K(dfcol,max_variazione):
    mean=dfcol.mean()
    sigma=dfcol.std()
    for alpha in np.linspace(0,6,601):
        if mlab.normpdf(alpha*sigma,mean,sigma)<=0.0001:
            break
    max_scostamento=alpha*sigma
    K=max_variazione/max_scostamento
    return K

def norm_elo(x):
    e=round(max(1450,x))
    return int(e)

KG=calc_K(elo["scostamento"],70)
KS=calc_K(elo["scostamento squadra"],20)


elo["elo nuovo"]=elo["elo precedente"]+KG*elo["scostamento"]+KS*elo["scostamento squadra"]
elo["elo nuovo"]=elo["elo nuovo"].apply(norm_elo)

elo[["elo nuovo"]].sort_values("elo nuovo",ascending=False)

,elo nuovo
giocatore,
Ella F.,1567
Kuno,1557
ITG,1548
Niko,1531
Joyopi,1526
queffe,1515
Eudes,1512
Bango Skank,1507
Plata,1506


In [32]:
a=gironi[gironi["A"]!="(riposo)"][["A"]].copy()
a.columns=["squadra"]
a["girone"]="A"
a

b=gironi[gironi["B"]!="(riposo)"][["B"]].copy()
b.columns=["squadra"]
b["girone"]="B"
b

c=gironi[gironi["C"]!="(riposo)"][["C"]].copy()
c.columns=["squadra"]
c["girone"]="C"
c

q=a.append(b).append(c).set_index("squadra")
q=q.merge(squadre,left_index=True,right_index=True,how="outer")
q=q.reset_index().set_index("giocatore")
q

elo_g=elo.merge(q,how="outer",
                left_index=True,right_index=True)
elo_g=elo_g[["squadra_y","elo nuovo"]].fillna(1500).sort_values("elo nuovo",
                                                                           ascending=False)
elo_g["elo nuovo"]=pd.to_numeric(elo_g["elo nuovo"],downcast="signed")
elo_g.columns=["squadra","elo"]
elo_g

,squadra,elo
giocatore,,
Ella F.,Armata Brancaleone,1567
Kuno,D Euforiche,1557
ITG,the 3 BrOthers,1548
Niko,I Bananari,1531
Joyopi,Green House,1526
queffe,D Euforiche,1515
Eudes,Retroguardia Trash,1512
Bango Skank,Green House,1507
Plata,Green House,1506


In [33]:
mu.to_table_html(elo_g,with_index=True)

<table style='margin:auto;' border='1' width='90%'>
<tr><th>&nbsp;</th>
<th>squadra</th>
<th>elo</th>
</tr>
<tr>
<th>Ella F.</th>
<td style='text-align:center;'>Armata Brancaleone</td>
<td style='text-align:center;'>1567</td>
</tr>
<tr>
<th>Kuno</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>1557</td>
</tr>
<tr>
<th>ITG</th>
<td style='text-align:center;'>the 3 BrOthers</td>
<td style='text-align:center;'>1548</td>
</tr>
<tr>
<th>Niko</th>
<td style='text-align:center;'>I Bananari</td>
<td style='text-align:center;'>1531</td>
</tr>
<tr>
<th>Joyopi</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>1526</td>
</tr>
<tr>
<th>queffe</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>1515</td>
</tr>
<tr>
<th>Eudes</th>
<td style='text-align:center;'>Retroguardia Trash</td>
<td style='text-align:center;'>1512</td>
</tr>
<tr>
<th>Bango Skank</th>
<td style='text-align:center;'>Green House</t

## Salvataggio dei risultati

In [34]:
## CSV
label="giornata_%d" % GIORNATA

# formazioni
formazioni_csv="data/formazioni_"+label+".csv"
formazioni.to_csv(formazioni_csv,index_label="squadra")

# accoppiamenti
accoppiamenti_csv="data/accoppiamenti_"+label+".csv"
accoppiamenti.to_csv(accoppiamenti_csv)

# voti
voti_csv="data/voti_"+label+".csv"
voti.to_csv(voti_csv)

# risultati
risultati_csv="data/risultati_"+label+".csv"
risultati.to_csv(risultati_csv)

# schedina
schedina_csv="data/schedina_"+label+".csv"
schedina.to_csv(schedina_csv,index=False)

# classifica
classifica_csv="data/classifica_"+label+".csv"
classifica.to_csv(classifica_csv)

# elo
elo_csv="data/elo_"+label+".csv"
elo.to_csv(elo_csv)

# elo_g
elo_g_csv="data/elo_g_"+label+".csv"
elo_g.to_csv(elo_g_csv)

# elo_s
elo_s_csv="data/elo_s_"+label+".csv"
elo_s.to_csv(elo_s_csv)

# ammissioni
ammissioni_csv="data/ammissioni_"+label+".csv"
ammissioni.to_csv(ammissioni_csv)


In [35]:
x=accoppiamenti.reset_index()
x.as_matrix()


array([['A', 1, 'D Euforiche', 'Kuno', 'Marcello', 'Kuno', 'queffe'],
       ['A', 1, 'Gli ultimi', 'crazycat', 'Ghigo', 'caipiroska', 'crazycat'],
       ['B', 1, 'Green House', 'Bango Skank', 'Bango Skank', 'Plata',
        'Joyopi'],
       ['B', 1, 'Gli scrittori del quartierino', 'Marty12', 'Federico72',
        'camparino', 'Marty12'],
       ['B', 2, 'I Bananari', 'simone volponi', 'massimopud',
        'simone volponi', 'Niko'],
       ['B', 2, 'the 3 BrOthers', 'AryaSophia', 'ettore70', 'ITG',
        'AryaSophia'],
       ['C', 1, 'Living Stones', 'Macleo', 'Vincenzo Iennaco', 'Ljuset',
        'Macleo'],
       ['C', 1, 'Le TreMende', 'Emy', 'sefora', 'Thea', 'Emy'],
       ['C', 2, 'Retroguardia Trash', 'Andrea28', 'AlexComan', 'Andrea28',
        'Eudes'],
       ['C', 2, 'Armata Brancaleone', 'AdStr', 'Skorpiòs', 'Ella F.',
        'AdStr']], dtype=object)

In [36]:
elo_g

,squadra,elo
giocatore,,
Ella F.,Armata Brancaleone,1567
Kuno,D Euforiche,1557
ITG,the 3 BrOthers,1548
Niko,I Bananari,1531
Joyopi,Green House,1526
queffe,D Euforiche,1515
Eudes,Retroguardia Trash,1512
Bango Skank,Green House,1507
Plata,Green House,1506
